In [1]:
# Dependencies
import requests
import pandas as pd
import matplotlib as plt
import numpy as np
import json
import sqlalchemy
from sqlalchemy import create_engine
from pymongo import MongoClient

In [2]:
# Save url as variable
url = "https://us-states.p.rapidapi.com/all"

# Set headers
headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "us-states.p.rapidapi.com"
}

# Make a GET request and save in variable
response = requests.request("GET", url, headers=headers)
data = response.json()

# Print the response
print(response.text)

[{"name": "Alabama", "postal": "AL", "capital": {"name": "Montgomery", "latitude": "32.377716", "longitude": "-86.300568"}, "date": "Dec 14, 1819", "population": {"density_km": "37", "total": "5024279", "density_mi": "95"}, "nickname": "Heart of Dixie", "fips": "01", "demonym": "Alabamian", "cities": [{"name": "Huntsville", "population": "215006"}, {"name": "Birmingham", "population": "200733"}, {"name": "Montgomery", "population": "198956"}, {"name": "Mobile", "population": "189572"}, {"name": "Tuscaloosa", "population": "101113"}], "lowest_point": "Gulf of Mexico", "highest_point": "Cheaha Mountain", "elevation": {"min_ft": "sea level", "min_m": "sea level", "mean_ft": "500", "max_rank": "38", "max_ft": "2413", "span_ft": "2413", "mean_rank": "44", "span_m": "736", "mean_m": "150", "max_m": "736"}, "area": {"total_mi": "52420.07", "land_rank": "28", "land_km": "131171", "water_rank": "25", "total_rank": "30", "land_mi": "50645.33", "water_km": "4597", "total_km": "135767", "land_perc

In [3]:
# Write the data to a json file
with open('data.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

In [4]:
# Create a pandas data frame
df = pd.json_normalize(response.json())
df

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,...,symbols.handgun,symbols.hymn,symbols.star,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,American Samoa,AS,"Feb 20, 1929",Motu o Fiafiaga,60,American Samoan,"[{'name': 'Tafuna', 'population': '7945'}, {'n...",Pacific Ocean,Lata Mountain on Ta‘ū island,https://www.americansamoa.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Colorado,CO,"Aug 1, 1876",Centennial State,08,Coloradan,"[{'name': 'Denver', 'population': '719238'}, {...",Arikaree River at Kansas border,Mount Elbert,http://www.colorado.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Connecticut,CT,"Jan 9, 1788",Constitution State,09,Connecticuter,"[{'name': 'Bridgeport', 'population': '144900'...",Long Island Sound,Massachusetts border on the southern slope of ...,https://portal.ct.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Delaware,DE,"Dec 7, 1787",First State,10,Delawarean,"[{'name': 'Wilmington', 'population': '70635'}...",Atlantic Ocean,Near the Ebright Azimuth,https://delaware.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Florida,FL,"Mar 3, 1845",Sunshine State,12,Floridian,"[{'name': 'Jacksonville', 'population': '90795...",Atlantic Ocean and Gulf of Mexico,Britton Hill,http://myflorida.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Check data types
df.dtypes

name                          object
postal                        object
date                          object
nickname                      object
fips                          object
                               ...  
symbols.heritage livestock    object
symbols.bat                   object
symbols.oyster                object
symbols.cocktail              object
symbols.domestic animal       object
Length: 203, dtype: object

In [6]:
# Check the counts
df.count()

name                          56
postal                        56
date                          56
nickname                      56
fips                          56
                              ..
symbols.heritage livestock     1
symbols.bat                    1
symbols.oyster                 1
symbols.cocktail               1
symbols.domestic animal        1
Length: 203, dtype: int64

In [7]:
# Slice the data frame to check for nulls
df.iloc[:,196:203]

,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Drop all territories
states_df = df.drop([2,11,36,41,47])
states_df

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,...,symbols.handgun,symbols.hymn,symbols.star,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Colorado,CO,"Aug 1, 1876",Centennial State,08,Coloradan,"[{'name': 'Denver', 'population': '719238'}, {...",Arikaree River at Kansas border,Mount Elbert,http://www.colorado.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Connecticut,CT,"Jan 9, 1788",Constitution State,09,Connecticuter,"[{'name': 'Bridgeport', 'population': '144900'...",Long Island Sound,Massachusetts border on the southern slope of ...,https://portal.ct.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Delaware,DE,"Dec 7, 1787",First State,10,Delawarean,"[{'name': 'Wilmington', 'population': '70635'}...",Atlantic Ocean,Near the Ebright Azimuth,https://delaware.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Florida,FL,"Mar 3, 1845",Sunshine State,12,Floridian,"[{'name': 'Jacksonville', 'population': '90795...",Atlantic Ocean and Gulf of Mexico,Britton Hill,http://myflorida.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Georgia,GA,"Jan 2, 1788",Peach State,13,Georgian,"[{'name': 'Atlanta', 'population': '504527'}, ...",Atlantic Ocean,Brasstown Bald,https://www.georgia.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Drop half of the columns that have nulls
clean_states_df = states_df.drop(columns=['status','other_nicknames','standard_federal_region','status','other_nicknames','standard_federal_region','koppen_climate','population.density_km','elevation.min_ft','elevation.min_m','elevation.mean_ft','elevation.span_ft','elevation.mean_rank','elevation.span_m','elevation.mean_m','elevation.max_m','area.land_km','area.water_rank','area.total_rank','area.land_mi','area.water_km','area.total_km','area.land_percent','area.water_mi','area.water_percent','symbols.mascot','symbols.fruit','symbols.tree fruit','symbols.soil','symbols.game bird','symbols.reptile','symbols.amphibian','symbols.nut','symbols.vegetable','symbols.rock','symbols.saltwater fish','symbols.mammal','symbols.butterfly','symbols.cake','symbols.insect','symbols.mineral','symbols.spirit','symbols.fossil','symbols.freshwater fish','symbols.wildflower','symbols.shell','symbols.land mammal','symbols.fish','symbols.bolt action rifle','symbols.dog','symbols.marine mammal','symbols.sport','symbols.plant','symbols.metal','symbols.colors','symbols.drink','symbols.firearm','symbols.dinosaur','symbols.floral emblem','symbols.folk dance','symbols.grape','symbols.fruit and vegetable','symbols.musical instrument','symbols.primitive fish','symbols.grain','symbols.cooking pot','symbols.lichen','symbols.grass','symbols.marine reptile','symbols.dance','symbols.tartan','symbols.gold rush ghost town', 'symbols.marine animal','symbols.macroinvertebrate','symbols.wildlife animal','symbols.dessert','symbols.bug','symbols.beverage','symbols.herb','symbols.saltwater mammal','symbols.heritage cattle breed','symbols.pie','symbols.stone','symbols.tortoise','symbols.saltwater reptile','symbols.prepared food','symbols.waltz','symbols.cold water gamefish','symbols.crop','symbols.raptor','symbols.exercise','symbols.artifact','symbols.snackfood','symbols.prairie grass','symbols.slogan','symbols.river','symbols.rifle','symbols.red wine grape','symbols.white wine grape','symbols.flying fossil','symbols.gun', 'symbols.wild animal game species',	'symbols.sports car', 'symbols.crustacean',	'symbols.meat pie','symbols.berry',	'symbols.treat','symbols.ballad','symbols.cat','symbols.team sport','symbols.donut','symbols.bean',	'symbols.cookie','symbols.muffin','symbols.folk song', 'symbols.bee','symbols.mushroom', 'symbols.waterfowl','symbols.toy','symbols.tree nut','symbols.fruit tree','symbols.hockey team','symbols.football team','symbols.endangered species','symbols.soft drink','symbols.poultry','symbols.saltwater game fish','symbols.freshwater game fish','symbols.spider','symbols.cryptid','symbols.bush','symbols.freshwater trout','symbols.salamander', 'symbols.marsupial','symbols.flying mammal','symbols.game animal','symbols.microbe','symbols.coral','symbols.appetizer','symbols.heritage horse','symbols.jewelry','symbols.bread','symbols.shrub','symbols.pastry','symbols.dish','symbols.handgun','symbols.hymn', 'symbols.astronomical symbol','symbols.emblem','symbols.heritage livestock','symbols.bat','symbols.oyster','symbols.cocktail','symbols.domestic animal','symbols.hero','symbols.shellfish',"symbols.children's flower",'symbols.heroine','symbols.polka','symbols.music','symbols.game mammal','symbols.aquatic animal','symbols.frog','symbols.star','symbols.horse','symbols.neckwear','symbols.silver rush ghost town','symbols.marine fish','symbols.fabric',	'symbols.pets',	'symbols.cactus','symbols.animal','population.density_mi','ap_abbreviation','gpo_abbreviation'])
clean_states_df

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,...,elevation.max_ft,area.total_mi,area.land_rank,census_bureau.region,census_bureau.division,symbols.gem,symbols.bird,symbols.song,symbols.tree,symbols.flower
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,...,2413,52420.07,28,South,East South Central,Star Blue Quartz,Yellowhammer,Alabama,Southern Longleaf Pine,Camellia
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,...,20310,665384.04,1,West,Pacific,Jade,Willow Ptarmigan,Alaska's Flag,Sitka Spruce,Forget-Me-Not
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,...,12637,113990.3,6,West,Mountain,Turquoise,Cactus Wren,"Arizona March Song,Arizona",Palo Verde,Saguaro Cactus Blossom
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,...,2753,53178.55,27,South,West South Central,Diamond,Mockingbird,NaN,Pine Tree,NaN
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,...,14505,163694.74,3,West,Pacific,Benitoite,California Quail,NaN,"Coast Redwood,Giant Sequoia",California Poppy
6,Colorado,CO,"Aug 1, 1876",Centennial State,08,Coloradan,"[{'name': 'Denver', 'population': '719238'}, {...",Arikaree River at Kansas border,Mount Elbert,http://www.colorado.gov/,...,14440,104093.67,8,West,Mountain,Aquamarine,Lark Bunting,"Where the Columbines Grow,Rocky Mountain High",Colorado Blue Spruce,Rocky Mountain Columbine
7,Connecticut,CT,"Jan 9, 1788",Constitution State,09,Connecticuter,"[{'name': 'Bridgeport', 'population': '144900'...",Long Island Sound,Massachusetts border on the southern slope of ...,https://portal.ct.gov/,...,2379,5543.41,48,Northeast,New England,NaN,American Robin,Yankee Doodle,Charter Oak,Mountain Laurel
8,Delaware,DE,"Dec 7, 1787",First State,10,Delawarean,"[{'name': 'Wilmington', 'population': '70635'}...",Atlantic Ocean,Near the Ebright Azimuth,https://delaware.gov/,...,447,2488.72,50,South,South Atlantic,NaN,Delaware Blue Hen,Our Delaware,American Holly,Peach Blossoms
9,Florida,FL,"Mar 3, 1845",Sunshine State,12,Floridian,"[{'name': 'Jacksonville', 'population': '90795...",Atlantic Ocean and Gulf of Mexico,Britton Hill,http://myflorida.com/,...,345,65757.7,26,South,South Atlantic,Moonstone,Northern Mockingbird,Our Folks at Home,Sabal Palm,Orang Blossom
10,Georgia,GA,"Jan 2, 1788",Peach State,13,Georgian,"[{'name': 'Atlanta', 'population': '504527'}, ...",Atlantic Ocean,Brasstown Bald,https://www.georgia.gov/,...,4784,59425.15,21,South,South Atlantic,Quartz,Brown Thrasher,Georgia on My Mind,Southern Live Oak,Cherokee Rose


In [28]:
# Show all columns of data frame
pd.options.display.max_columns = None
clean_states_df.to_html('states.html',index=False,classes=["table","table-striped","table-bordered"])
clean_states_df

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,per_capita_income,median_household_income,representatives,time_zones,motto,subdivisions,capital.name,capital.latitude,capital.longitude,population.total,elevation.max_rank,elevation.max_ft,area.total_mi,area.land_rank,census_bureau.region,census_bureau.division,symbols.gem,symbols.bird,symbols.song,symbols.tree,symbols.flower
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,"$23,606","$49,861",7,[CST],Audemus jura nostra defendere,"[Autauga, Baldwin, Barbour, Bibb, Blount, Bull...",Montgomery,32.377716,-86.300568,5024279,38,2413,52420.07,28,South,East South Central,Star Blue Quartz,Yellowhammer,Alabama,Southern Longleaf Pine,Camellia
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,"$33,062","$74,346",1,"[AKST, HST]",North to the Future,"[Aleutians East Borough, Aleutians West Census...",Juneau,58.301598,-134.420212,733391,1,20310,665384.04,1,West,Pacific,Jade,Willow Ptarmigan,Alaska's Flag,Sitka Spruce,Forget-Me-Not
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,"$25,715","$59,246",9,[MST],Ditat Deus,"[Apache, Cochise, Coconino, Gila, Graham, Gree...",Phoenix,33.448143,-112.096962,7151502,12,12637,113990.3,6,West,Mountain,Turquoise,Cactus Wren,"Arizona March Song,Arizona",Palo Verde,Saguaro Cactus Blossom
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,"$22,883","$47,062",4,[CST],Regnat populus,"[Arkansas, Ashley, Baxter, Benton, Boone, Brad...",Little Rock,34.746613,-92.288986,3011524,37,2753,53178.55,27,South,West South Central,Diamond,Mockingbird,NaN,Pine Tree,NaN
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,"$30,441","$75,277",53,[PST],Eureka,"[Alameda, Alpine, Amador, Butte, Calaveras, Co...",Sacramento,38.576668,-121.493629,39538223,2,14505,163694.74,3,West,Pacific,Benitoite,California Quail,NaN,"Coast Redwood,Giant Sequoia",California Poppy
6,Colorado,CO,"Aug 1, 1876",Centennial State,08,Coloradan,"[{'name': 'Denver', 'population': '719238'}, {...",Arikaree River at Kansas border,Mount Elbert,http://www.colorado.gov/,"$32,357","$71,953",7,[MST],Nil sine numine,"[Adams, Alamosa, Arapahoe, Archuleta, Baca, Be...",Denver,39.739227,-104.984856,5773714,3,14440,104093.67,8,West,Mountain,Aquamarine,Lark Bunting,"Where the Columbines Grow,Rocky Mountain High",Colorado Blue Spruce,Rocky Mountain Columbine
7,Connecticut,CT,"Jan 9, 1788",Constitution State,09,Connecticuter,"[{'name': 'Bridgeport', 'population': '144900'...",Long Island Sound,Massachusetts border on the southern slope of ...,https://portal.ct.gov/,"$39,373","$76,348",5,[EST],Qui transtulit sustinet,"[Fairfield, Hartford, Litchfield, Middlesex, N...",Hartford,41.764046,-72.682198,3605944,39,2379,5543.41,48,Northeast,New England,NaN,American Robin,Yankee Doodle,Charter Oak,Mountain Laurel
8,Delaware,DE,"Dec 7, 1787",First State,10,Delawarean,"[{'name': 'Wilmington', 'population': '70635'}...",Atlantic Ocean,Near the Ebright Azimuth,https://delaware.gov/,"$30,488","$64,805",1,[EST],Liberty and independence,"[Kent, New Castle, Sussex]",Dover,39.157307,-75.519722,989948,54,447,2488.72,50,South,South Atlantic,NaN,Delaware Blue Hen,Our Delaware,American Holly,Peach Blossoms
9,Florida,FL,"Mar 3, 1845",Sunshine State,12,Floridian,"[{'name': 'Jacksonville', 'population': '90795...",Atlantic Ocean and Gulf of Mexico,Britton Hill,http://myflorida.com/,"

In [ ]:
# Save url as variable
url = "https://jsearch.p.rapidapi.com/search"

# Set querystring 
querystring = {"query":"Entry Level Data Scientist in USA","num_pages":"20"}

# Set headers
headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

# Make a GET request and save in variable
response = requests.request("GET", url, headers=headers, params=querystring)
entry_level_data_scientists = response.json()

# Write the data to a json file
with open('entry_level_data_scientists.json', 'w') as outfile:
    json.dump(entry_level_data_scientists, outfile, indent=4)


# Print response
print(response.text)

In [ ]:
# Save url as variable
url = "https://jsearch.p.rapidapi.com/search"

# Set querystring
querystring = {"query":"Entry Level Data Analyst in USA","num_pages":"20"}

# Set headers
headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

# Make a GET request and save in variable
response = requests.request("GET", url, headers=headers, params=querystring)
entry_level_data_analyst = response.json()

# Write the data to a json file
with open('entry_level_data_analyst.json', 'w') as outfile:
    json.dump(entry_level_data_analyst, outfile, indent=4)


# Print response
print(response.text)

In [13]:
# Select certain columns to be used and turn json file into a pandas data frame 
columns = ["employer_name","job_title","job_apply_link","job_description","job_is_remote","job_city","job_state","job_country","job_latitude","job_longitude"]
df = pd.json_normalize(entry_level_data_analyst, 'data')
new_df = df.loc[:, columns]
new_df

,employer_name,job_title,job_apply_link,job_description,job_is_remote,job_city,job_state,job_country,job_latitude,job_longitude
0,Jobcertify,Entry level - Data Analyst I (Remote),https://www.linkedin.com/jobs/view/entry-level...,The Data Analyst I is responsible for the mana...,True,Indianapolis,IN,US,39.768402,-86.158066
1,SynergisticIT,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Normal,IL,US,40.514202,-88.990630
2,THE wallmart,Entry level Data Analyst / Full time (Remote),https://www.recruit.net/job/data-analyst-jobs/...,Data analyst responsibilities include conducti...,False,San Jose,CA,US,37.338740,-121.885254
3,Attitude Remote Jobs,Entry Level - Research Data Analyst (Remote),https://www.linkedin.com/jobs/view/entry-level...,We are currently seeking an entry-level Resear...,True,Los Angeles,CA,US,34.052235,-118.243680
4,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.ziprecruiter.com/c/Staffigo-Techni...,About us\n\nStaffigo is in the business of pro...,False,Roswell,GA,US,34.023243,-84.361560
...,...,...,...,...,...,...,...,...,...,...
68,United States Army,Entry Level Data Analyst â US Army (13J),https://fountainvalley-ca.geebo.com/jobs-onlin...,Are your math and data skills ready for a chal...,False,Fountain Valley,CA,US,33.709023,-117.954410
69,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://maryland.tarta.ai/j/PNU2LoABCr2uuunQDN...,About Us\n\nStaffigo is in the business of pro...,False,Annapolis,MD,US,38.978447,-76.492180
70,United States Army,Entry Level Data Analyst â US Army (13J),https://southgate-ca.geebo.com/jobs-online/vie...,Are your math and data skills ready for a chal...,False,South Gate,CA,US,33.954740,-118.212010
71,THE wallmart,Entry level Data Analyst / Full time (Remote),https://www.whatjobs.com/jobs/Entry-level-Data...,Data analyst responsibilities include conducti...,False,San Jose,IL,US,40.305600,-89.602880


In [14]:
# Select certain columns to be used and turn json file into a pandas data frame 
columns = ["employer_name","job_title","job_apply_link","job_description","job_is_remote","job_city","job_state","job_country","job_latitude","job_longitude"]
df2 = pd.json_normalize(entry_level_data_scientists, 'data')
new_data_df = df2.loc[:, columns]
new_data_df

,employer_name,job_title,job_apply_link,job_description,job_is_remote,job_city,job_state,job_country,job_latitude,job_longitude
0,National Security Agency,Data Scientist - Entry to Expert Level (Maryla...,https://www.linkedin.com/jobs/view/data-scient...,Responsibilities\n\nData science at the Nation...,False,Fort Meade,MD,US,39.104263,-76.734240
1,Insight Global,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,Our large trucking and leasing client is hirin...,False,Reading,PA,US,40.335648,-75.926870
2,SynergisticIT,Entry Level Data Scientist,https://www.linkedin.com/jobs/view/entry-level...,"At SynergisticIT, we aim to bring aboard IT pr...",False,Seattle,WA,US,47.606210,-122.332070
3,Jobcertify,Entry level - Data Analyst I (Remote),https://www.linkedin.com/jobs/view/entry-level...,The Data Analyst I is responsible for the mana...,True,Indianapolis,IN,US,39.768402,-86.158066
4,SynergisticIT,Entry Level Data Scientist,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Lansing,MI,US,42.732536,-84.555534
5,SynergisticIT,Entry Level Data Scientist (Remote),https://www.datayoshi.com/offer/672278/entry-l...,Synergistic IT is a full-service staffing and ...,False,College Station,TX,US,30.627977,-96.334404
6,SynergisticIT,Entry Level Data Scientist,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Portland,ME,US,43.659100,-70.256820
7,SynergisticIT,ENTRY LEVEL DATA SCIENTIST,https://www.linkedin.com/jobs/view/entry-level...,SYNERGISTICIT wants every candidate to know th...,False,Oklahoma City,OK,US,35.467560,-97.516426
8,I28 Technologies,Entry Level Data Scientist-333325,https://www.glassdoor.com/job-listing/entry-le...,"• Analysis tools, defining, and implementing i...",False,Marysville,MI,US,42.912530,-82.486860
9,SynergisticIT,Junior Level Data Scientist,https://www.ziprecruiter.com/c/SynergisticIT/J...,"Job Description\n\nGreetings,\nSynergistic IT ...",False,Fremont,CA,US,37.548540,-121.988590


In [27]:
# Add the two data frames together 
result_df = pd.concat([new_df, new_data_df], axis=0)
result_df.to_html('jobs.html',index=False,classes=["table","table-striped","table-bordered"])
result_df

,employer_name,job_title,job_apply_link,job_description,job_is_remote,job_city,job_state,job_country,job_latitude,job_longitude
0,Jobcertify,Entry level - Data Analyst I (Remote),https://www.linkedin.com/jobs/view/entry-level...,The Data Analyst I is responsible for the mana...,True,Indianapolis,IN,US,39.768402,-86.158066
1,SynergisticIT,Entry Level Data Analyst,https://www.linkedin.com/jobs/view/entry-level...,About Us\n\nSynergistic IT is a full-service s...,False,Normal,IL,US,40.514202,-88.990630
2,THE wallmart,Entry level Data Analyst / Full time (Remote),https://www.recruit.net/job/data-analyst-jobs/...,Data analyst responsibilities include conducti...,False,San Jose,CA,US,37.338740,-121.885254
3,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.learn4good.com/jobs/aurora/illinoi...,About us\n\nStaffigo is in the business of pro...,False,Aurora,IL,US,41.760586,-88.320070
4,"Staffigo Technical Services, LLC",Entry Level Data Analyst,https://www.ziprecruiter.com/c/Staffigo-Techni...,About us\n\nStaffigo is in the business of pro...,False,Roswell,GA,US,34.023243,-84.361560
...,...,...,...,...,...,...,...,...,...,...
47,SynergisticIT,Entry level data analyst /Data scientist/Jr Ja...,https://www.whatjobs.com/jobs/Entry-level-data...,SYNERGISTICIT wants every candidate to know th...,False,Newark,NJ,US,40.735657,-74.172360
48,Unistaff Inc.,Data Engineer - Entry Level,https://www.glassdoor.com/job-listing/data-eng...,Data Entry Candidates needed in the Elk Grove ...,False,Elk Grove Village,IL,US,42.003918,-87.970345
49,RecruitTalent,Mid-Level Data Scientist,https://www.linkedin.com/jobs/view/mid-level-d...,"As a Mid-Level Data Engineer, this role focuse...",False,Herndon,VA,US,38.969555,-77.386100
50,PCS GLOBAL TECH,Data Engineer Entry Level (Bootcamp),https://www.glassdoor.com/job-listing/data-eng...,Data Engineer Responsibilities:\n• Liaising wi...,False,Los Angeles,CA,US,34.052235,-118.243680


In [16]:
# Connect to the MongoDB server
client = MongoClient()

# Choose the database you want to use
db = client.data

# Insert the clean_states_df data frame into a collection named "clean_states"
db.states.insert_many(clean_states_df.to_dict('records'))

# Insert the result_df data frame into a collection named "result"
db.jobs.insert_many(result_df.to_dict('records'))